In [1]:
import json
import os
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

# y_pred
parent_path = os.path.dirname(os.getcwd())
#fp = ".\out_model\pred_test.jsonl"   # output of Tabbie
fp = parent_path + "/tabbie/evaluation/small-100-shuffled/pred_test.jsonl"
with open(fp, 'r') as json_file:
    json_list = list(json_file)

# initiate a dataframe
df_pred = pd.DataFrame(columns=['fname','col_id','cluster_id'])

for j in json_list:
    result = json.loads(j)
    fname = result['table_id']
    col_ids = result['col_labels']

    for i in col_ids:
        df_pred = df_pred.append({'fname': fname, 'col_id': i, 'cluster_id': result['header'][i]}, ignore_index=True)
df_pred.sort_values(['fname', 'col_id'], ascending=[True, True], inplace=True)

selection_dict = {}
keys = df_pred['fname'].unique()
for key in keys:
    value = df_pred[df_pred['fname']==key]['col_id'].tolist()
    selection_dict[key] = value

# y_true
parent_path = os.path.dirname(os.getcwd())
#test_table_path = parent_path + "/src/data/LocalBusiness/Splitting_TabbieData/Train_Test/test tables"   #TO CHANGE
#test_label_path = parent_path + "/src/data/LocalBusiness/Splitting_TabbieData/Train_Test"         #TO CHANGE

test_table_path = "../../../src/data/schemafiltereddata/tabbie/without-100/test"   #TO CHANGE
test_label_path = "../../../src/data/schemafiltereddata/tabbie/without-100"

# initiate a dataframe
df_true = pd.DataFrame(columns=['fname','col_id','cluster_id'])

#test_labels = pd.read_csv(os.path.join(test_label_path, 'test_label.csv'))
#test_labels = pd.read_csv(os.path.join(test_label_path, 'test_label_new.csv'))
test_labels = pd.read_csv(os.path.join(test_label_path, 'test_label_without-100.csv'))
if 'Unnamed: 0' in test_labels.columns:
    test_labels.drop(columns=['Unnamed: 0'], inplace= True)

tables = test_labels['fname'].unique()
for t in tables:
    tmp = pd.read_csv(os.path.join(test_table_path, t)) # change to read_json...
    
    fname = os.path.splitext(t)[0]
    for i in range(len(tmp.columns)):
        df_true = df_true.append({'fname':fname, 'col_id': i, 'cluster_id': tmp.columns[i]}, ignore_index=True)

# filter for including only predicted instances
df_true2 = pd.DataFrame(columns=['fname','col_id','cluster_id'])

for key in selection_dict.keys():
    value = selection_dict[key]
    tmp = df_true[df_true['fname']== key]
    tmp2 = tmp[tmp['col_id'].isin(value)]
    df_true2 = pd.concat([df_true2, tmp2], ignore_index=True)
df_true2.sort_values(['fname', 'col_id'], ascending=[True, True], inplace=True)

# Evaluation
from sklearn.metrics import accuracy_score, f1_score
y_true = df_true2['cluster_id'].astype(str)
y_pred = df_pred['cluster_id'].astype(str)

print("accuracy: ", accuracy_score(y_true, y_pred))
print("micro f1: ", f1_score(y_true, y_pred, average='micro'))
print("macro f1: ", f1_score(y_true, y_pred, average='macro'))
print("weighted f1: ", f1_score(y_true, y_pred, average='weighted'))


accuracy:  1.0
micro f1:  1.0
macro f1:  1.0
weighted f1:  1.0


In [8]:
df_pred['cluster_id'].equals(df_true2['cluster_id'])

False

In [19]:
df_pred['cluster_id'].values

array(['name', 'numberofpages', 'datepublished', ..., 'partofseries',
       'name', 'partofseries'], dtype=object)

In [9]:
df_pred['cluster_id']

34374             name
34375    numberofpages
34376    datepublished
34377        publisher
34378       bookformat
             ...      
46033     partofseries
25976             name
25977     partofseries
43128             name
43129     partofseries
Name: cluster_id, Length: 50709, dtype: object

In [13]:
df_true2

,fname,col_id,cluster_id
0,Book_4thestate.co.uk_September2020,0,name
1,Book_4thestate.co.uk_September2020,1,numberofpages
2,Book_4thestate.co.uk_September2020,2,datepublished
3,Book_4thestate.co.uk_September2020,3,publisher
4,Book_4thestate.co.uk_September2020,4,bookformat
...,...,...,...
50704,TVEpisode_watermarkthefilm.com_September2020,1,partofseries
50705,TVEpisode_ww2d.org_September2020,0,name
50706,TVEpisode_ww2d.org_September2020,1,partofseries
50707,TVEpisode_zebrahead.org_September2020,0,name


In [15]:
df_true2['cluster_id'].values

array(['name', 'numberofpages', 'datepublished', ..., 'partofseries',
       'name', 'partofseries'], dtype=object)

In [17]:
vergleich = pd.DataFrame({'A': df_pred['cluster_id'].values, 'B': df_true2['cluster_id'].values}, )

In [18]:
vergleich.loc[~(vergleich['A'] == vergleich['B'])]


,A,B


In [5]:
df_true2

,fname,col_id,cluster_id
0,Book_4thestate.co.uk_September2020,0,name
1,Book_4thestate.co.uk_September2020,1,numberofpages
2,Book_4thestate.co.uk_September2020,2,datepublished
3,Book_4thestate.co.uk_September2020,3,publisher
4,Book_4thestate.co.uk_September2020,4,bookformat
...,...,...,...
50704,TVEpisode_watermarkthefilm.com_September2020,1,partofseries
50705,TVEpisode_ww2d.org_September2020,0,name
50706,TVEpisode_ww2d.org_September2020,1,partofseries
50707,TVEpisode_zebrahead.org_September2020,0,name


In [ ]:

test_table_path = "../../../src/data/schemafiltereddata/tabbie/test"   #TO CHANGE
test_label_path = "../../../src/data/schemafiltereddata/tabbie"

# initiate a dataframe
df_true = pd.DataFrame(columns=['fname','col_id','cluster_id'])

#test_labels = pd.read_csv(os.path.join(test_label_path, 'test_label.csv'))
test_labels = pd.read_csv(os.path.join(test_label_path, 'test_label_new.csv'))

In [ ]:
tables = test_labels['fname'].unique()
for t in tables:
    tmp = pd.read_csv(os.path.join(test_table_path, t)) 
    fname = os.path.splitext(t)[0]
    for i in range(len(tmp.columns)):
        df_true = df_true.append({'fname':fname, 'col_id': i, 'cluster_id': tmp.columns[i]}, ignore_index=True)
